# <span style="color:blue">GDM V.2 (*Python Version*)</span>


#### This Glacio-hydrological Degree-day Model (GDM) V.2 calculates river discharge with four contributing components (Snowmelt, Icemelt, Rainfall and Baseflow).

# LULC
    1. Forest
    2. Crops
    3. Grassland
    4. bareland/clouds
    5. Water/flooded vegetation
    6. Settlement
    7. Debris Covered Glacier
    8. Clean Glacier
        
# Snowmelt and icemelt, $M$ (mm/day) 
$ \displaystyle M = (k_s or k_b) \times T \, \mathrm{ if }\, T > 0 $   
$ \displaystyle M = 0  \, \mathrm{ if }\, T \leq 0 $   
$ \displaystyle M = (k_d \times T \, \mathrm{ if }\, T > 0 $   
$T$: air temperature ($^\circ $C)  
$k_s$: degree day factor (mm/$^\circ $C/day) for snow   
$k_b$: degree day factor (mm/$^\circ $C/day) for bare ice   
$k_d$: degree day factor (mm/$^\circ $C/day) for debris-covered ice    

# Potential Evapotraspiration

The Hargreaves method estimates PET based on the following equation:

$$ PET = 0.0023\times(T_{mean} + 17.8)\times Ra \times 0.0408 \times \sqrt{T_{max} - T_{min}} $$

$PET$: Potential evapotranspiration in mm/day

$T_{mean}$: Average daily temperature in degrees Celsius

$T_{max}$: Maximum daily temperature in degrees Celsius

$T_{min}$ Minimum daily temperature in degrees Celsius

$Ra$ is the extraterrestrial radiation in $$MJ/m^{2}/day$$

# Baseflow

\begin{align*}
W_{\text{seep}} &= \text{ifthenelse}( \text{total_water} > \text{PET}, \text{Atotal_water} - \text{runoff}, 0 ) \\
W_{\text{rch}} &= (1 - \exp(-1/\text{delta_gwsh})) \times W_{\text{seep}} + \exp(-1/\text{delta_gwsh}) \times W_{\text{rch}} \\
W_{\text{seep_dp}} &= \text{beta_dp} \times W_{\text{rch}} \\
W_{\text{rch_sh}} &= W_{\text{rch}} - W_{\text{seep_dp}} \\
Q_{\text{b_sh}} &= Q_{\text{b_sh}} \times \exp(-\text{alpha_gwsh} \times 1) + W_{\text{rch_sh}} \times (1 - \exp(-\text{alpha_gwsh} \times 1)) \\
W_{\text{rch_dp}} &= (1 - \exp(-1/\text{delta_gwdp})) \times W_{\text{seep_dp}} + \exp(-1/\text{delta_gwdp}) \times W_{\text{rch_dp}} \\
Q_{\text{b_dp}} &= Q_{\text{b_dp}} \times \exp(-\text{alpha_gwdp} \times 1) + W_{\text{rch_dp}} \times (1 - \exp(-\text{alpha_gwdp} \times 1)) \\
Q_{\text{b}} &= Q_{\text{b_sh}} + Q_{\text{b_dp}} \quad \text{(Base Flow contribution)}
\end{align*}

$W_{seep}$: Total amount of water exiting the bottom of the soil profile

**Atotal_water**: Total Available water after reducing $PET$

$W_{rch}$: Amount of recharge entering the aquifers

**delta_gwsh**: Delay time of the overlying geologic formations (days)

**beta_dp**: Coefficient of shallow aquifer percolation to deep aquifer

**delta_gwdp**: Delay time or drainage time of the deep aquifer geologic formations (days)

**alpha_gwsh, alpha_gwdp**: Recession constants

**Wseep_dp**:  total amount of water exiting the bottom of the shallow aquifer

**Qb_dp**: Baseflow component from deep aquifer.

**Qb_sh**:  Baseflow component from the shallow aquifer



# Reference
http://www.ncgia.ucsb.edu/SANTA_FE_CD-ROM/sf_papers/wesseling_cees/santa_fe.html



## Main Model

## Run the cell below.

In [2]:
from pcraster import *
from pcraster.framework import *
import numpy as np
import pandas as pd
import datetime
import io
import tempfile
# Loading param1 sheet from param file.
param1 = pd.read_excel('param.xlsx', index_col='Parameters').squeeze().to_dict()

# extract values of parameters using their names
Start_date, End_date, Start_timesteps, End_timesteps, lat_deg, Delta_gwsh, Delta_gwdp, Alpha_gwsh,\
Alpha_gwdp,Beta_dp, kx, tc, Kd = \
    (param1.get(param) for param in ['start_date', 'end_date', 'start_timesteps',\
                                     'end_timesteps', 'lat_deg', 'delta_gwsh', 'delta_gwdp', \
                                     'alpha_gwsh', 'alpha_gwdp', 'beta_dp', \
                                       'Kx','Tc', 'kd'])
#loading param2 sheet from param file
param2 = pd.read_excel('param.xlsx',skiprows=1,sheet_name="Param2")

# Setting timesteps,Date and adding Julian day, later required in the main model 
start_date = pd.to_datetime(Start_date)                              
end_date = pd.to_datetime(End_date)                                  
start_timesteps = Start_timesteps                                   
end_timesteps = End_timesteps                                                                         
time_steps = pd.DataFrame({"Date": pd.date_range(start_date, end_date)}) 
time_steps["Month"] = time_steps.Date.dt.month
time_steps["Timestep"]= range(start_timesteps,end_timesteps+1)
time_steps.set_index("Timestep", inplace=True)
# Add Julian day column
time_steps["JulianDay"] = time_steps["Date"].apply(lambda x: x.timetuple().tm_yday)

# creating temporary lookup tables for runoff coefficient and interception threshold
Runoff_coeffs=[]
interception_threshold=[]
landuseid=[1,2,3,4,5,6,7,8]
# read the land use map
landuse = readmap("./Inputs/lulc")
for i in range(12):
    # create an in-memory text file
    with io.StringIO() as text_file:
        # write text to the file
        for Id in landuseid:
            text_file.write(str(Id)+" "+str(param2["Rc_L"+str(Id)][i])+"\n")

        # write the contents of the text file to a temporary file on disk
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as tmp_file1:
            tmp_file1.write(text_file.getvalue())
    Runoff_coeffs.append(lookupscalar(tmp_file1.name, landuse))
    os.unlink(tmp_file1.name)
    with io.StringIO() as text_file:
        # write text to the file
        for Id in landuseid:
            text_file.write(str(Id)+" "+str(param2["Ic_L"+str(Id)][i])+"\n")
            
        # write the contents of the text file to a temporary file on disk
        with tempfile.NamedTemporaryFile(mode='w', delete=False) as tmp_file2:
            tmp_file2.write(text_file.getvalue())
    interception_threshold.append(lookupscalar(tmp_file2.name, landuse))
    os.unlink(tmp_file2.name)

#latitude in degree 
latitude_deg = lat_deg 

# Baseflow parameters
delta_gwsh = Delta_gwsh
delta_gwdp = Delta_gwdp  
alpha_gwsh = Alpha_gwsh  
alpha_gwdp = Alpha_gwdp  
beta_dp    = Beta_dp     
class RunoffModel(DynamicModel):
    def __init__(self, cloneMap):
        DynamicModel.__init__(self)
        setclone(cloneMap)
    
    def initial(self):
        # Add here the static maps that we need to read from disk
        self.landuse = self.readmap("./Inputs/lulc") 

        self.DEM = self.readmap("./Inputs/dem")      

        self.mask = self.readmap("./Inputs/mask")    
        
        # input for degree day factors, runoff coefficient and interception threshold
        self.Kb=np.array(param2["Kb"])    
        self.Ks=np.array(param2["Ks"])     
        self.kd = scalar(Kd)               
    
        self.Runoff_coeffs = Runoff_coeffs 
        
        self.interception_threshold= interception_threshold
        
        #initializing incremental timestep 
        self.IncrementalTimestep= time_steps.index[0]
        
        # other input parameters
        self.DEBRIS = scalar(7)     
        self.BARE = scalar(8)         
        self.ConvConst = float(celllength()**2)/(1000*86400)
        self.Kx= kx                        
        
        # initializations
        self.Snow = self.landuse * 0.0 
        self.W_rch = self.landuse * 0.0 
        self.Qb_sh = self.landuse * 0.0 
        self.W_rch_dp = self.landuse * 0.0  
        self.Qb_dp = self.landuse * 0.0 
        self.SnowMelt0 = self.DEM*0.0  
        self.IceMelt0 = self.DEM*0.0 
        self.Rain0 = self.DEM*0.0 
        
        # Critical Temperature
        self.Tc = scalar(tc)              
        
        # Add here the calculation of the flow direction map
        self.flowdirection = lddcreate(self.DEM,1e31,1e31,1e31,1e31)
        self.flowdirection = lddmask(self.flowdirection,self.mask)
        self.report(self.flowdirection,"ldd") 
        #self.flowdirection=self.readmap("ldd")
        
        # initialise time series output
        self.Measurements = self.readmap("Measurementid")
        DischargeAtMeasurementLocations = "discharge.tss" 
        self.DischargeTSS = TimeoutputTimeseries(DischargeAtMeasurementLocations,self,self.Measurements,noHeader=False)
        ####
        RainAtMeasurementLocations = "Rain.tss" 
        self.RainTSS = TimeoutputTimeseries(RainAtMeasurementLocations,self,self.Measurements,noHeader=False)
        ####
        IcemeltAtMeasurementLocations = "Icemelt.tss" 
        self.IcemeltTSS = TimeoutputTimeseries(IcemeltAtMeasurementLocations,self,self.Measurements,noHeader=False)
        ####
        SnowmeltAtMeasurementLocations = "Snowmelt.tss" 
        self.SnowmeltTSS = TimeoutputTimeseries(SnowmeltAtMeasurementLocations,self,self.Measurements,noHeader=False)
        ####
        BaseflowAtMeasurementLocations = "Baseflow.tss" 
        self.BaseflowTSS = TimeoutputTimeseries(BaseflowAtMeasurementLocations,self,self.Measurements,noHeader=False)
        
    def dynamic(self):
        Timestep=int(self.IncrementalTimestep)
        
        # importing degree day factor, runoff coefficient, interception threshold from initial section
        ks=self.Ks[time_steps["Month"][Timestep]-1]
        kb=self.Kb[time_steps["Month"][Timestep]-1] 
        runoff_coeff=self.Runoff_coeffs[time_steps["Month"][Timestep]-1]
        interception=self.interception_threshold[time_steps["Month"][Timestep]-1]
        
        # importing precpitation input data
        Precipitation = self.readmap("./Prec/prec") 
        
        # importing Temperature input data
        Tmax=self.readmap("./Tmax/tmax")  
        Tmin=self.readmap("./Tmin/tmin")  
        
        # Average temperature
        Temperature = (Tmax+Tmin)/2     
        #self.report(Temperature,"./temperature/temp")
        
        #Calculating interception
        Interception = min(Precipitation,interception) 
        #self.report(Interception,"./intercp/intercp")
        
        # calculating net precipitation
        NetPrecipitation = Precipitation - Interception
        
        #Calculation of Potential Evapotraspiration using Hargreaves method
        # Calculate extraterrestrial radiation (Ra)
        julian_day = time_steps["JulianDay"][Timestep]
        doy_factor = 2 * np.pi / 365
        dr = 1 + 0.033 * np.cos(doy_factor * julian_day)
        solar_declination = 0.409 * np.sin(doy_factor * julian_day - 1.405)
        latitude_rad = np.radians(latitude_deg)
        sunset_hour_angle = np.arccos(-np.tan(latitude_rad) * np.tan(solar_declination))
        extraterrestrial_rad = (24 * 60 / np.pi) * 0.082 * dr * \
        (sunset_hour_angle * np.sin(latitude_rad) * np.sin(solar_declination)\
         + np.cos(latitude_rad) * np.cos(solar_declination) * np.sin(sunset_hour_angle))
        PET = max(0.0023 * extraterrestrial_rad *0.408* ((Tmax - Tmin) ** 0.5) * (Temperature + 17.8),0)    
        #self.report(PET,"PET/PET")
        
        #PrecipitationPET= max(NetPrecipitation-PET,0)
        #self.report(PrecipitationPET,"./PrecipitationPET/PrecPET")
        
        # Calculating rain and snow
        Rain0 = ifthenelse(Temperature >  self.Tc, NetPrecipitation, 0.0)
        self.Snow += ifthenelse(Temperature <= self.Tc, NetPrecipitation, 0.0)
        # self.report(self.Snow,"./Data1/snow")
        #self.report(Rain0,"./rain/rain")
        
        # Calculate melting
        IceMelt0 = ifthenelse(self.landuse == self.BARE, ifthenelse(Temperature > 0, kb * Temperature, 0), 0)
        SnowMelt0 = ifthenelse(self.Snow > 0, ifthenelse(Temperature > 0, ks * Temperature, 0), 0)
        IceMelt0 += ifthenelse(self.landuse == self.DEBRIS, ifthenelse(Temperature > 0, self.kd * Temperature, 0), 0)
        self.Snow = max(self.Snow - SnowMelt0, 0)      
        #self.report(SnowMelt0,"./snowmelt/smelt")
        #self.report(IceMelt0,"./icemelt/imelt")
        #self.report(self.Snow,"./snow/r")
        
        #Snowmelt, Icemelt, Rain after PET
        f=ifthenelse((IceMelt0+SnowMelt0+Rain0)>0,PET/(IceMelt0+SnowMelt0+Rain0),0)
        SnowMelt= max(SnowMelt0- SnowMelt0*f,0)
        IceMelt= max(IceMelt0- IceMelt0*f,0)
        Rain= max(Rain0 - Rain0*f,0)
        #Total Water
        TotalWater= SnowMelt+IceMelt+Rain
        
        #Surface Runoff
        SnowMeltRunoff = (SnowMelt*runoff_coeff)
        IceMeltRunoff =(IceMelt*runoff_coeff)
        RainRunoff = (Rain*runoff_coeff)
        SurfaceRunoff= SnowMeltRunoff+IceMeltRunoff+RainRunoff
        
        ## Seeping water
        W_seep = TotalWater - SurfaceRunoff
        # self.report(W_seep,"./Data1/W_seep")
    
        # Calculate baseflow
        self.W_rch = (1 - np.exp(-1/delta_gwsh)) * W_seep + np.exp(-1/delta_gwsh) * self.W_rch
        self.W_seep_dp = beta_dp * self.W_rch
        self.W_rch_sh = self.W_rch - self.W_seep_dp
        self.Qb_sh = self.Qb_sh * np.exp(-alpha_gwsh*1) + self.W_rch_sh * (1 - np.exp(-alpha_gwsh*1))
        self.W_rch_dp = (1 - np.exp(-1/delta_gwdp)) * self.W_seep_dp + np.exp(-1/delta_gwdp) * self.W_rch_dp
        self.Qb_dp = self.Qb_dp * np.exp(-alpha_gwdp*1) + self.W_rch_dp * (1 - np.exp(-alpha_gwdp*1))
        # Base Flow contribution
        Qb = self.Qb_sh + self.Qb_dp 
        #self.report(Qb,"./baseflow/Qb")      
        
        #calculating the accumulated  amount of water flowing through the river basin and converting mm/day to m3/sec
        SnowMeltFlow= accuflux(self.flowdirection, SnowMeltRunoff)*self.ConvConst
        IceMeltFlow= accuflux(self.flowdirection,IceMeltRunoff)*self.ConvConst
        RainFlow= accuflux(self.flowdirection, RainRunoff)*self.ConvConst
        BaseFlow =accuflux(self.flowdirection, Qb) *self.ConvConst## Base flow contribution
        
        SnowMeltContribution=((1 - self.Kx) * SnowMeltFlow + self.Kx * self.SnowMelt0)
        IceMeltContribution=((1 - self.Kx) * IceMeltFlow + self.Kx * self.IceMelt0)
        RainContribution=((1 - self.Kx) * RainFlow + self.Kx * self.Rain0)
        
        Qtot= (SnowMeltContribution+IceMeltContribution+RainContribution+BaseFlow)
        
        # Report discharge,Baseflow,Rain Contribution, Icemelt Contribution, SnowMelt Contribution time series at measurement locations
        self.BaseflowTSS.sample(BaseFlow)
        self.RainTSS.sample(RainContribution)
        self.IcemeltTSS.sample(IceMeltContribution)
        self.SnowmeltTSS.sample(SnowMeltContribution)
        self.DischargeTSS.sample(Qtot)
        self.report(Qtot,"./Qtot/qtot")
        
        #updating SnowMelt0, IceMelt0, Rain0
        self.SnowMelt0 = SnowMeltContribution
        self.IceMelt0=IceMeltContribution
        self.Rain0=RainContribution

        self.IncrementalTimestep= self.IncrementalTimestep+1
        
myModel = RunoffModel("./Inputs/mask.map")
dynModelFw = DynamicFramework(myModel, lastTimeStep=int(time_steps.index[-1]), firstTimestep=int(time_steps.index[0]))
dynModelFw.run()

aguila("discharge.tss")


........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

KeyboardInterrupt: 

##### Visualizing the discharge output using aguila

In [7]:
from pcraster import *
aguila("discharge.tss")
